In [1]:
import os
import json

In [2]:
JSONINFILENAME = "032018_annotations_new.json"

with open(JSONINFILENAME, "r") as read_file:
    JSONdata = json.load(read_file)

In [3]:
CLASSES = ["clumped", "isolated"]

SINGLE_OUTPUT_CLASS = True
SINGLE_CLASS = "plant" # To use if SINGLE_OUTPUT_CLASS is True

SWAP_IMG_FILE_FORMAT = True
NEW_IMG_FORMAT = ".png" # To use if SWAP_IMG_FILE_FORMAT is True

In [4]:
CWD = os.getcwd()

EXPORTDIR = os.path.join(CWD, "pascalvoc_labels_patched" + ("_oneclass" if SINGLE_OUTPUT_CLASS else ""))

if not os.path.exists(EXPORTDIR):
    os.makedirs(EXPORTDIR)
    print("WARNING: Making output dir, {}".format(EXPORTDIR))

In [5]:
for filename, data in JSONdata.items():
    if SWAP_IMG_FILE_FORMAT:
        filename = os.path.splitext(os.path.basename(filename))[0] + NEW_IMG_FORMAT
    print(f"***** PROCESSING {filename} *****")
    for class_ in CLASSES:
        print(f"{class_}: {data[class_]}")

***** PROCESSING 12_G0428471_0.png *****
clumped: []
isolated: [[232, 1, 266, 9], [218, 67, 255, 126], [218, 215, 250, 245], [219, 438, 255, 499]]
***** PROCESSING 12_G0428471_1.png *****
clumped: []
isolated: [[219, 0, 255, 5], [236, 87, 263, 126], [230, 266, 258, 338], [226, 455, 277, 499]]
***** PROCESSING 12_G0428471_2.png *****
clumped: []
isolated: [[226, 0, 277, 19], [240, 318, 288, 355], [249, 419, 274, 458]]
***** PROCESSING 12_G0428471_3.png *****
clumped: []
isolated: [[236, 23, 262, 56], [234, 210, 265, 244], [234, 287, 265, 330], [230, 370, 277, 410]]
***** PROCESSING 12_G0428471_4.png *****
clumped: []
isolated: [[232, 70, 277, 109], [237, 185, 259, 230], [224, 364, 252, 386], [226, 406, 257, 439], [226, 487, 259, 499]]
***** PROCESSING 12_G0428471_5.png *****
clumped: []
isolated: [[226, 0, 259, 32], [227, 65, 247, 94], [227, 152, 251, 169], [238, 202, 257, 215]]
***** PROCESSING 12_G0428471_6.png *****
clumped: []
isolated: []
***** PROCESSING 12_G0428471_7.png *****
cl

In [6]:
WIDTH  = 500
HEIGHT = 500
CHANNELS = 3

In [7]:
CLASSNAME = "plant"

In [8]:
INPUTDIR = "D:\\tensorflow\\projects\\PlantClumpDetection\\git\\datasets\\annotations\\img_patches_8"+("_"+NEW_IMG_FORMAT[NEW_IMG_FORMAT.rfind('.')+1:] if SWAP_IMG_FILE_FORMAT else "") #os.path.join(CWD, "img"+("_"+NEW_IMG_FORMAT[NEW_IMG_FORMAT.rfind('.')+1:] if SWAP_IMG_FILE_FORMAT else "")) # WARNING: Using convention in simpleImageFileFormatConverter.py; remove the '+' in brackets, need be
DATABASE = "annotations"
SEGMENTED = 0

In [9]:
'''import xml.dom.minidom as minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

print(prettify(root))'''

def indent(elem, level=0, more_sibs=False, tab="    "):
    i = "\n"
    if level:
        i += (level-1) * tab
    num_kids = len(elem)
    if num_kids:
        if not elem.text or not elem.text.strip():
            elem.text = i + tab
            if level:
                elem.text += tab
        count = 0
        for kid in elem:
            indent(kid, level+1, count < num_kids - 1)
            count += 1
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
            if more_sibs:
                elem.tail += tab
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i
            if more_sibs:
                elem.tail += tab

In [10]:
def addPascalObjectToET(placeToAdd, class_, T, L, B, R):
    object_ = ET.SubElement(placeToAdd, "object")
    name = ET.SubElement(object_, "name").text = class_
    pose = ET.SubElement(object_, "pose").text = "Unspecified"
    truncated = ET.SubElement(object_, "truncated").text = str(0)
    difficult = ET.SubElement(object_, "difficult").text = str(0)
    
    bndbox = ET.SubElement(object_, "bndbox")
    xmin = ET.SubElement(bndbox, "xmin").text = str(L)
    ymin = ET.SubElement(bndbox, "ymin").text = str(T)
    xmax = ET.SubElement(bndbox, "xmax").text = str(R)
    ymax = ET.SubElement(bndbox, "ymax").text = str(B)

In [11]:
import xml.etree.cElementTree as ET

for filename, data in JSONdata.items():
    print("***** PROCESSING {}{} *****".format(filename, f" as {NEW_IMG_FORMAT}" if SWAP_IMG_FILE_FORMAT else ""))
    
    basename = os.path.splitext(os.path.basename(filename))[0]
    
    outFilename = basename + ".txt"
    
    filename = filename if not SWAP_IMG_FILE_FORMAT else basename + NEW_IMG_FORMAT
    
    # Create XML (all but bounding boxes)

    root = ET.Element("annotation")
    folder = ET.SubElement(root, "folder").text = INPUTDIR
    filename = ET.SubElement(root, "filename").text = filename
    path = ET.SubElement(root, "path").text = os.path.join(INPUTDIR, filename)

    source = ET.SubElement(root, "source")
    database = ET.SubElement(source, "database").text = DATABASE

    size = ET.SubElement(root, "size")
    sWidth = ET.SubElement(size, "width").text = str(WIDTH)
    sHeight = ET.SubElement(size, "height").text = str(HEIGHT)
    sDepth = ET.SubElement(size, "depth").text = str(CHANNELS)

    segmented = ET.SubElement(root, "segmented").text = str(SEGMENTED)
    
    # Create bounding boxes 
    
    for class_ in CLASSES:
        tempData = data[class_]
        classIdx = CLASSES.index(class_)
        for datumToWrite in tempData:
            T = datumToWrite[0]
            L = datumToWrite[1]
            B = datumToWrite[2]
            R = datumToWrite[3]
    
            addPascalObjectToET(root, (SINGLE_CLASS if SINGLE_OUTPUT_CLASS else class_), T, L, B, R)
    
    # Create tree
    
    tree = ET.ElementTree(root)
    
    # Export tree
    
    tempRoot = tree.getroot()
    indent(tempRoot)
    
    outFilename = os.path.splitext(os.path.basename(filename))[0] + ".xml"
    tree.write(open(os.path.join(EXPORTDIR, outFilename), 'w'), encoding='unicode')
    

***** PROCESSING 12_G0428471_0.tif as .png *****
***** PROCESSING 12_G0428471_1.tif as .png *****
***** PROCESSING 12_G0428471_2.tif as .png *****
***** PROCESSING 12_G0428471_3.tif as .png *****
***** PROCESSING 12_G0428471_4.tif as .png *****
***** PROCESSING 12_G0428471_5.tif as .png *****
***** PROCESSING 12_G0428471_6.tif as .png *****
***** PROCESSING 12_G0428471_7.tif as .png *****
***** PROCESSING 4_G0428450_0.tif as .png *****
***** PROCESSING 4_G0428450_1.tif as .png *****
***** PROCESSING 4_G0428450_2.tif as .png *****
***** PROCESSING 4_G0428450_3.tif as .png *****
***** PROCESSING 4_G0428450_4.tif as .png *****
***** PROCESSING 4_G0428450_5.tif as .png *****
***** PROCESSING 4_G0428450_6.tif as .png *****
***** PROCESSING 4_G0428450_7.tif as .png *****
***** PROCESSING 34_G0428510_0.tif as .png *****
***** PROCESSING 34_G0428510_1.tif as .png *****
***** PROCESSING 34_G0428510_2.tif as .png *****
***** PROCESSING 34_G0428510_3.tif as .png *****
***** PROCESSING 34_G0428510

***** PROCESSING 43_G0428534_7.tif as .png *****
***** PROCESSING 19_G0428483_0.tif as .png *****
***** PROCESSING 19_G0428483_1.tif as .png *****
***** PROCESSING 19_G0428483_2.tif as .png *****
***** PROCESSING 19_G0428483_3.tif as .png *****
***** PROCESSING 19_G0428483_4.tif as .png *****
***** PROCESSING 19_G0428483_5.tif as .png *****
***** PROCESSING 19_G0428483_6.tif as .png *****
***** PROCESSING 19_G0428483_7.tif as .png *****
***** PROCESSING 40_G0428528_0.tif as .png *****
***** PROCESSING 40_G0428528_1.tif as .png *****
***** PROCESSING 40_G0428528_2.tif as .png *****
***** PROCESSING 40_G0428528_3.tif as .png *****
***** PROCESSING 40_G0428528_4.tif as .png *****
***** PROCESSING 40_G0428528_5.tif as .png *****
***** PROCESSING 40_G0428528_6.tif as .png *****
***** PROCESSING 40_G0428528_7.tif as .png *****
***** PROCESSING 5_G0428453_0.tif as .png *****
***** PROCESSING 5_G0428453_1.tif as .png *****
***** PROCESSING 5_G0428453_2.tif as .png *****
***** PROCESSING 5_G042

***** PROCESSING 13_G0448814_1.tif as .png *****
***** PROCESSING 13_G0448814_2.tif as .png *****
***** PROCESSING 13_G0448814_3.tif as .png *****
***** PROCESSING 13_G0448814_4.tif as .png *****
***** PROCESSING 13_G0448814_5.tif as .png *****
***** PROCESSING 13_G0448814_6.tif as .png *****
***** PROCESSING 13_G0448814_7.tif as .png *****
***** PROCESSING 19_G0448829_0.tif as .png *****
***** PROCESSING 19_G0448829_1.tif as .png *****
***** PROCESSING 19_G0448829_2.tif as .png *****
***** PROCESSING 19_G0448829_3.tif as .png *****
***** PROCESSING 19_G0448829_4.tif as .png *****
***** PROCESSING 19_G0448829_5.tif as .png *****
***** PROCESSING 19_G0448829_6.tif as .png *****
***** PROCESSING 19_G0448829_7.tif as .png *****
***** PROCESSING 23_G0448836_0.tif as .png *****
***** PROCESSING 23_G0448836_1.tif as .png *****
***** PROCESSING 23_G0448836_2.tif as .png *****
***** PROCESSING 23_G0448836_3.tif as .png *****
***** PROCESSING 23_G0448836_4.tif as .png *****
***** PROCESSING 23_